# Adding Dependeces

In [6]:
using DelimitedFiles
using ProgressMeter
using NearestNeighbors
include("../../src/forces.jl")
include("../../src/Neighbor.jl")

knn_cu (generic function with 1 method)

# Initial Conditions

In [7]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
t_f = 100000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 16

# Forces Conditions
r_max = 3.5
s = 2
K = 1

# Coding COnditions
n_text = 4

4

# Init Data

In [8]:
# Initial Data
X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end])

X_f = zeros(size(X,1))
for i in 1:size(X_f,1)
    A = sqrt(X[i,1]^2+X[i,2]^2+X[i,3]^2)
    if A > 0.8*R_agg
        X_f[i] = 1
    else
        X_f[i] = 2
    end
end
X_f_2 = Int.(vcat(X_f,X_f));

# Fusioning tho Spheres

In [9]:
Size = Float32((findmax(X[:,1])[1] - findmin(X[:,1])[1])/2 + 1)
X_2 = vcat(X,X)
X_2[1:size(X,1),1] = X_2[1:size(X,1),1] .- (Size)
X_2[size(X,1):end,1] = X_2[size(X,1):end,1] .+ Size

X = X_2;

# Forces (CPU)

In [10]:
# # Calculating position of every cell on the fusion
# p = Progress(Int(t_f/dt),barlen=25)
# for i in 0:Int(t_f/dt)
#     if mod(i, t_f/n_text/dt) == 0
#         open("CPU/tf_($(t_f))|dt_($(dt))|rm_($(r_max))|s=($(s))|K_($(K))_CPU.xyz", "a") do f
#             write(f, "$(size(X, 1))\n")
#             write(f, "t=$(i*dt)\n")
#             writedlm(f,hcat(X_f_2, X), ' ')
#         end
#     end

#     if mod(i, n_knn) == 0 || i != 0
#         # Calculating kNN
#         kdtree = KDTree(X')
#         idxs = Int.(zeros(14,size(X,1)))
#         for i in 1:size(X)[1]
#             # Scan neighbours
#             idxs[:,i], _ = knn(kdtree, X[i,1:3], 14, true)
#             # println(knn(kdtree, X[i,1:3], 14, true)[1])
#             # println(idxs[:,i],)
#         end
#         idxs = Matrix(idxs)
#     end

#     # Calculating Forces
#     dX = force(X, idxs, r_max, s, K)

#     global X = X + dX*dt
    
#     next!(p)
# end

# Forces (GPU)

In [11]:
# Inizializate Variables for kNN
X = X |> cu

i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
idx = hcat([[CartesianIndex(i,1) for i=1:nn] for j=1:size(X,1)]...) |> cu

# Inizializate Variables for Forces
r = zeros(nn,size(X)[1],3) |> cu
dist = zeros(nn, size(X)[1]) |> cu
F = zeros(nn, size(X)[1],3) |> cu
dX = zeros(size(X)[1],3) |> cu


# Calculating position of every cell on the fusion
p = Progress(Int(t_f/dt),barlen=25)
for i in 0:Int(t_f/dt)
    if mod(i, Int(t_f/n_text/dt)) == 0
        X_w = Matrix(X)
        open("GPU/tf_($(t_f))|dt_($(dt))|rm_($(r_max))|s=($(s))|K_($(K))_GPU.xyz", "a") do f
            write(f, "$(size(X, 1))\n")
            write(f, "t=$(i*dt)\n")
            writedlm(f,hcat(X_f_2, X_w), ' ')
        end
    end

    if mod(i, n_knn) == 0 || i != 0
        # Calculating kNN
        knn_cu()
    end

    # Calculating Forces
    forces_cu(r_max, s, K)

    global X = X + dX*dt
    next!(p)
end

Progress:   1%|▏                        |  ETA: 4:19:26m39m

LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
     [1] [0m[1mpoptask[22m[0m[1m([22m[90mW[39m::[0mBase.InvasiveLinkedListSynchronized[90m{Task}[39m[0m[1m)[22m
    [90m   @ [39m[90mBase[39m [90m./[39m[90;4mtask.jl:795[0m
     [2] [0m[1mwait[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[90mBase[39m [90m./[39m[90;4mtask.jl:804[0m
     [3] [0m[1mwait[22m[0m[1m([22m[90mc[39m::[0mBase.GenericCondition[90m{Base.Threads.SpinLock}[39m[0m[1m)[22m
    [90m   @ [39m[90mBase[39m [90m./[39m[90;4mcondition.jl:106[0m
     [4] [0m[1m_trywait[22m[0m[1m([22m[90mt[39m::[0mTimer[0m[1m)[22m
    [90m   @ [39m[90mBase[39m [90m./[39m[90;4masyncevent.jl:115[0m
     [5] [0m[1mwait[22m
    [90m   @ [39m[90m./[39m[90;4masyncevent.jl:133[0m[90m [inlined][39m
     [6] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m~/.julia/packages/CUDA/DfvRa/lib/cudadrv/[39m[90;4mst